In [ ]:
import torch
from sklearn.datasets import fetch_openml

from neural_blueprints.architectures import MLP
from neural_blueprints.config.architectures import MLPConfig
from neural_blueprints.config.utils import TrainerConfig
from neural_blueprints.config.components.composite.projections.input import TabularInputProjectionConfig
from neural_blueprints.config.components.composite.projections.output import TabularOutputProjectionConfig
from neural_blueprints.utils import Trainer, infer_types
from neural_blueprints.preprocess import TabularPreprocessor
from neural_blueprints.datasets import MaskedTabularDataset, TabularSingleLabelDataset

import logging
logging.basicConfig(
    level=logging.DEBUG,  # or DEBUG if you want even more detail
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [2]:
data = fetch_openml(name="adult", version=2, as_frame=True)
X = data.data
y = data.target

data = X.copy()
data['income'] = y

dtypes = infer_types(data)
data = data.astype(dtypes)
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
preprocessor = TabularPreprocessor()
data, discrete_features, continuous_features = preprocessor.run(data)
data.head()

2025-12-24 15:56:29,893 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 10 discrete features: ['workclass', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']
2025-12-24 15:56:29,893 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 5 continuous features: ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.109589,4,0.145129,2,14,5,7,4,3,2,0.000000,0.0,0.397959,39,1
1,0.287671,4,0.052451,12,16,3,5,1,5,2,0.000000,0.0,0.500000,39,1
2,0.150685,2,0.219649,8,4,3,11,1,5,2,0.000000,0.0,0.397959,39,2
3,0.369863,4,0.100153,16,2,3,7,1,3,2,0.076881,0.0,0.397959,39,2
4,0.013699,0,0.061708,16,2,5,0,4,5,1,0.000000,0.0,0.295918,39,1


### Income Inference Accuracy

In [4]:
dataset = TabularSingleLabelDataset(
    data=data,
    label_column='income',              # Specify the label column for single-label classification
    discrete_features=discrete_features,
    continuous_features=continuous_features
)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [5]:
# Define model configuration
mlp_config = MLPConfig(
    hidden_dims=[128, 64, 32, 16],
    normalization="batchnorm1d",
    activation='gelu',
    dropout_p=0.1,
    final_activation=None,
    input_projection=TabularInputProjectionConfig(
        cardinalities=dataset.cardinalities,
        hidden_dims=[64, 32],
        output_dim=[len(dataset.cardinalities)*16],
        normalization="batchnorm1d",
        activation="gelu",
        dropout_p=0.1
    ),
    output_projection=TabularOutputProjectionConfig(
        input_cardinalities=dataset.cardinalities,
        output_cardinalities=[3],
        input_dim=[len(dataset.cardinalities)*16],
        hidden_dims=[len(dataset.cardinalities)*8],
        normalization="batchnorm1d",
        activation="gelu",
        dropout_p=0.1
    )
)

# Initialize model
model = MLP(mlp_config)
model.blueprint()

2025-12-24 15:56:30,060 - neural_blueprints.architectures.mlp - INFO - Using input projection: TabularInputProjection
2025-12-24 15:56:30,062 - neural_blueprints.architectures.mlp - INFO - Using output projection: TabularOutputProjection


224
Sequential(
  (0): TabularInputProjection(
    (input_projections): ModuleList(
      (0): FeedForwardNetwork(
        (network): Sequential(
          (0): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=1, out_features=64, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropout): Dropout(p=0.1, inplace=False)
              )
            )
          )
          (1): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=64, out_features=32, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (d

MLPConfig(input_dim=224, hidden_dims=[128, 64, 32, 16], output_dim=224, normalization='batchnorm1d', activation='gelu', dropout_p=0.1, final_activation=None, input_projection=TabularInputProjectionConfig(cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42], hidden_dims=[64, 32], output_dim=[224], dropout_p=0.1, normalization='batchnorm1d', activation='gelu'), output_projection=TabularOutputProjectionConfig(input_cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42], output_cardinalities=[3], input_dim=[224], hidden_dims=[112], activation='gelu', normalization='batchnorm1d', dropout_p=0.1))

In [ ]:
trainer = Trainer(
    config=TrainerConfig(
        optimizer="adam",
        criterion="cross_entropy",
        learning_rate=1e-3,
        weight_decay=1e-5,
        batch_size=128,
        early_stopping_patience=5,
        save_weights_path="./models/mlp_adult.pth"
    ),
    model= model
)

# Train the model
trainer.train(train_dataset, val_dataset, epochs=20)

2025-12-24 15:56:30,392 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Directory ./models already exists. Existing weights are overwritten.


Training Epochs:   5%|▌         | 1/20 [00:02<00:49,  2.61s/epoch]

Epoch 1/20, Training Loss: 0.9279, Validation Loss: 0.9008


Training Epochs:  10%|█         | 2/20 [00:05<00:46,  2.60s/epoch]

...

Training Epochs:  65%|██████▌   | 13/20 [00:33<00:18,  2.58s/epoch]

Epoch 13/20, Training Loss: 0.8858, Validation Loss: 0.8783


Training Epochs:  70%|███████   | 14/20 [00:36<00:15,  2.57s/epoch]

Epoch 14/20, Training Loss: 0.8845, Validation Loss: 0.8772


2025-12-24 15:57:09,175 - neural_blueprints.utils.trainer - INFO - No improvement in validation loss for 5 consecutive epochs. Early stopping at epoch 15.
Training Epochs:  70%|███████   | 14/20 [00:38<00:16,  2.77s/epoch]
2025-12-24 15:57:09,176 - neural_blueprints.utils.trainer - INFO - Training completed in 38.78 seconds.
2025-12-24 15:57:09,176 - neural_blueprints.utils.trainer - INFO - Best validation loss: 8.7711e-01


In [7]:
trainer.predict(val_dataset)

2025-12-24 15:57:09,404 - neural_blueprints.utils.trainer - INFO - Inference completed in 0.04 seconds.


Predictions: tensor([2, 1, 1, 1, 1]), 
 Ground Truth: tensor([2, 1, 1, 1, 1])
Prediction Accuracy: 0.8673


np.float64(0.8673490276356193)

### Masked Dataset Inference Accuracy

In [8]:
dataset = MaskedTabularDataset(
    data = data,
    discrete_features = discrete_features,
    continuous_features = continuous_features,
    mask_prob=0.35
)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [9]:
# Define model configuration
mlp_config = MLPConfig(
    hidden_dims=[128, 64, 32, 16],
    normalization="batchnorm1d",
    activation='gelu',
    dropout_p=0.1,
    final_activation="relu",
    input_projection=TabularInputProjectionConfig(
        cardinalities=dataset.cardinalities,
        hidden_dims=[64, 32],
        output_dim=[len(dataset.cardinalities) * 16],
        normalization="batchnorm1d",
        activation="gelu",
        dropout_p=0.1
    ),
    output_projection=TabularOutputProjectionConfig(
        input_cardinalities=dataset.cardinalities,
        input_dim=[len(dataset.cardinalities)*16],
        hidden_dims=[8],
        activation="gelu",
        normalization="batchnorm1d",
        dropout_p=0.1
    )
)

# Initialize model
model = MLP(mlp_config)
model.blueprint()

2025-12-24 15:57:09,423 - neural_blueprints.architectures.mlp - INFO - Using input projection: TabularInputProjection
2025-12-24 15:57:09,425 - neural_blueprints.architectures.mlp - INFO - Using output projection: TabularOutputProjection


Sequential(
  (0): TabularInputProjection(
    (input_projections): ModuleList(
      (0): FeedForwardNetwork(
        (network): Sequential(
          (0): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=1, out_features=64, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropout): Dropout(p=0.1, inplace=False)
              )
            )
          )
          (1): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=64, out_features=32, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropo

MLPConfig(input_dim=240, hidden_dims=[128, 64, 32, 16], output_dim=240, normalization='batchnorm1d', activation='gelu', dropout_p=0.1, final_activation='relu', input_projection=TabularInputProjectionConfig(cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42, 2], hidden_dims=[64, 32], output_dim=[240], dropout_p=0.1, normalization='batchnorm1d', activation='gelu'), output_projection=TabularOutputProjectionConfig(input_cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42, 2], output_cardinalities=None, input_dim=[240], hidden_dims=[8], activation='gelu', normalization='batchnorm1d', dropout_p=0.1))

In [ ]:
trainer = Trainer(
    config=TrainerConfig(
        optimizer="adam",
        criterion="reconstruction",
        learning_rate=1e-3,
        weight_decay=1e-5,
        batch_size=128,
        early_stopping_patience=5,
        save_weights_path="./models/mlp_adult.pth"
    ),
    model= model
)

# Train the model
trainer.train(train_dataset, val_dataset, epochs=20)

2025-12-24 15:57:09,439 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Directory ./models already exists. Existing weights are overwritten.


Training Epochs:   5%|▌         | 1/20 [00:04<01:17,  4.06s/epoch]

Epoch 1/20, Training Loss: 218.7951, Validation Loss: 172.6412


Training Epochs:  10%|█         | 2/20 [00:08<01:12,  4.05s/epoch]

...

Training Epochs:  90%|█████████ | 18/20 [01:14<00:08,  4.08s/epoch]

Epoch 18/20, Training Loss: 165.9259, Validation Loss: 164.4827


Training Epochs:  95%|█████████▌| 19/20 [01:18<00:04,  4.07s/epoch]

Epoch 19/20, Training Loss: 165.8852, Validation Loss: 164.4942


2025-12-24 15:58:32,453 - neural_blueprints.utils.trainer - INFO - No improvement in validation loss for 5 consecutive epochs. Early stopping at epoch 20.
Training Epochs:  95%|█████████▌| 19/20 [01:23<00:04,  4.37s/epoch]
2025-12-24 15:58:32,454 - neural_blueprints.utils.trainer - INFO - Training completed in 83.01 seconds.
2025-12-24 15:58:32,454 - neural_blueprints.utils.trainer - INFO - Best validation loss: 1.6445e+02


In [11]:
trainer.predict(val_dataset)

2025-12-24 15:58:32,560 - neural_blueprints.utils.trainer - INFO - Inference completed in 0.05 seconds.



Feature Column 0:
Predicted attribute values: [0.32654315 0.29183584 0.36896768 0.3251175  0.33438522]
True attribute values: [0.12328767 0.369863   0.60273975 0.32876712 0.60273975]
Accuracy: 0.2241

Feature Column 1:
Predicted attribute values: [4 4 4 4 4]
True attribute values: [4. 4. 4. 6. 5.]
Accuracy: 0.6847

Feature Column 2:
Predicted attribute values: [6.8652357e-06 4.2349566e-06 1.7897803e-05 9.6128460e-06 6.5769214e-06]
True attribute values: [0.10014241 0.11132558 0.19932346 0.128193   0.04370973]
Accuracy: 0.1286

Feature Column 3:
Predicted attribute values: [12 16 16 12 12]
True attribute values: [ 2. 10. 16. 16. 12.]
Accuracy: 0.4503

Feature Column 4:
Predicted attribute values: [0 0 0 0 0]
True attribute values: [14.  2. 16.  6.  2.]
Accuracy: 0.0000

Feature Column 5:
Predicted attribute values: [5 5 5 3 3]
True attribute values: [4. 5. 5. 3. 3.]
Accuracy: 0.7153

Feature Column 6:
Predicted attribute values: [0 0 0 0 0]
True attribute values: [10.  1.  5.  7. 10.]


{'avg_discrete_accuracy': np.float64(0.5833088591612344),
 'avg_continuous_accuracy': np.float64(0.5521321922051811),
 'overall_avg_accuracy': np.float64(0.57291663684255)}